# Alerting and Detection Strategy (ADS) Template
## Goal
The goal is the intended purpose of the alert. It is a simple, plaintext description of the type of behavior you're attempting to detect in your ADS.
## Categorization
The categorization is a mapping of the ADS to the relevant entry in the MITRE Adversarial Tactics, Techniques, and Common Knowledge (ATT&CK) Framework. ATT&CK provides a language for various post-exploitation techniques and strategies that adversaries might use.
## Strategy Abstract
The strategy abstract is a high-level walkthrough of how the ADS functions. This describes what the alert is looking for, what technical data sources are used, any enrichment that occurs, and any false positive minimization steps.
## Technical Context
Technical Context provides detailed information and background needed for a responder to understand all components of the alert.
## Blind Spots and Assumptions
Blind Spots and Assumptions are the recognized issues, assumptions, and areas where an ADS may not fire.
## False Positives
False Positives are the known instances of an ADS misfiring due to a misconfiguration, idiosyncrasy in the environment, or other non-malicious scenario.
## Validation
Validation are the steps required to generate a representative true positive event which triggers this alert.
## Priority
Priority describes the various alerting levels that an ADS may be tagged with.
## Response
These are the general response steps in the event that this alert fired.
## Additional Resources
Additional Resources are any other internal, external, or technical references that may be useful for understanding the ADS.

## Appendices
### Tools and Resources
1. AWS GuardDuty
2. AWS Athena
3. Your SIEM Tool
4. Incident Response Platform (e.g., Tines, TheHive)
### Contact Information
1. Incident Response Team: [Contact Details]
2. AWS Support: [Contact Details]

## Lessons Learned
### Steps
1. Update incident response procedures and playbooks.
2. Conduct a retrospective of the incident.
3. New: Update training materials and conduct training sessions for the incident response team.

## Recovery
### Steps
1. Monitor the systems for signs of weaknesses.
2. Restore the EC2 instance from a known good backup.
3. New: Conduct a vulnerability assessment to ensure that the issue has been resolved.

## Eradication
### Steps
1. Analyze the root cause.
2. Patch or update the software.
3. New: Validate the patch or update to ensure it has been applied successfully.

## Containment
### Short-Term & Long-Term Actions
#### Short-Term: Isolate the affected EC2 instance from the network.
```python
import boto3

def isolate_ec2_instance(instance_id):
    # Isolate the affected EC2 instance from the network by disassociating its Elastic IP and modifying security groups.
    ec2 = boto3.client('ec2')
    # Disassociate Elastic IP if any
    try:
        response = ec2.describe_instances(InstanceIds=[instance_id])
        public_ip = response['Reservations'][0]['Instances'][0].get('PublicIpAddress', None)
        if public_ip:
            ec2.disassociate_address(PublicIp=public_ip)
            print(f'Successfully disassociated Elastic IP {public_ip} from instance {instance_id}.')
    except Exception as e:
        print(f'Error disassociating Elastic IP: {e}')
    # Modify Security Groups to an isolation group with no inbound or outbound rules
    try:
        ec2.modify_instance_attribute(InstanceId=instance_id, Groups=['sg-isolation-group-id'])
        print(f'Successfully modified security group for instance {instance_id} to isolation group.')
    except Exception as e:
        print(f'Error modifying security group: {e}')
```
#### Long-Term: Review and update IAM roles and security groups to follow the principle of least privilege.

## Investigation
### Objective
Understand the nature of the access and whether it was malicious.
### Athena Queries
#### Query 1: Identify Docker Socket Access Events
```sql
-- New: Added more specific fields for better readability
SELECT eventTime, eventName, sourceIPAddress, userAgent
FROM cloudtrail_logs
WHERE eventName = 'DockerSocketAccessed'
AND eventSource = 'runtime'
AND instanceId = '<Affected_Instance_ID>'
ORDER BY eventTime DESC;
```
#### Query 2: Identify User Activities
```sql
-- New: Added filtering by event types that are generally associated with privilege escalation
SELECT eventTime, eventName, eventSource, sourceIPAddress, userAgent
FROM cloudtrail_logs
WHERE userIdentity.arn = '<Affected_User_ARN>'
AND eventName IN ('AssumeRole', 'AddUserToGroup', 'AttachUserPolicy')
AND eventTime BETWEEN '<Start_Time>' AND '<End_Time>'
ORDER BY eventTime DESC;
```

## Initial Response
### Objective
Quickly assess the situation and determine the scope of the incident.
### Steps
1. Validate the GuardDuty alert in the Security Account AWS Console.
2. Identify the affected EC2 instance and its associated metadata tags.
3. Move into the investigation phase.

# Versioning and Metadata
## Version: 1.1
## Last Updated: 2023-09-11
## Author: @Christopher Watkins
## Reviewers: [Names of Reviewers]
----
# Table of Contents
1. [Initial Response](#Initial-Response)
2. [Investigation](#Investigation)
3. [Athena Queries](#Athena-Queries)
4. [Containment](#Containment)
5. [Eradication](#Eradication)
6. [Recovery](#Recovery)
7. [Lessons Learned](#Lessons-Learned)
8. [Appendices](#Appendices)